In [ ]:
from bs4 import BeautifulSoup
#from bs4 import SoupStrainer
#from collections import Counter
import re
import requests
import pandas as pd
from time import sleep
from selenium import webdriver
import time

In [ ]:
#URL = "https://www.amazon.com.br/OCTOO-UP-BL-Suporte-Uptable-Preto/product-reviews/B07BTC67VS/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2"
#URLx = "https://www.amazon.com.br/Smart-Lâmpada-Colors-Bivolt-Wi-FI/product-reviews/B088C4QZV2/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber="

URLx = 'https://www.amazon.com.br/OCTOO-UP-BL-Suporte-Uptable-Preto/product-reviews/B07BTC67VS/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='

start_page = 1
last_page = 500

reviews_list = []
date_list = []
star_list = []

driver = webdriver.Edge()

for n_url in range(start_page, last_page+1):
    URL = URLx + str(n_url)
    #page = requests.get(URL)
    driver.get(URL)
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, 'html.parser')
    
    # Captura das reviews
    av = soup.select("span.a-size-base.review-text.review-text-content")

    for item in av:
        if item.find('span') != None:   #in item:
            text = item.span.get_text()
            text = re.sub('\n', '', text, 2)
            text = re.sub(' ', '', text, 2)
        else:
            text = ''
        reviews_list.append(text)
    
    # Captura das estrelas
    star = soup.select('a.a-link-normal span.a-icon-alt')

    for item in star:
        text = item.get_text()
        star_list.append(text[:3])
    
    # Captura das datas
    date = soup.select("span.a-size-base.a-color-secondary.review-date")

    for i,item in enumerate(date):
        if i > 1:
            text = item.get_text()
            text = re.sub('Avaliado no Brasil em ', '', text, 1)
            date_list.append(text)

driver.close()

d = {'reviews':reviews_list, 'star':star_list, 'date':date_list}
df_reviews = pd.DataFrame(data=d)

print('Shape:', df_reviews.shape)

Shape: (5000, 3)


In [ ]:
# Quantidade de reviews sem conteúdo
print('N° de reviews sem conteúdo:', df_reviews[df_reviews['reviews'] == '']['reviews'].count())

N° de reviews sem conteúdo: 48


In [ ]:
df_reviews

,reviews,star,date
0,"Produto de boa qualidade, tanto pela aparência...","4,0",24 de setembro de 2019
1,O suporte possui hastes grossas em ferro pinta...,"5,0",5 de março de 2020
2,Não recomendo!!Produto veio torto e não tem um...,"1,0",5 de fevereiro de 2020
3,,"4,0",14 de julho de 2020
4,Um dos braços articuláveis veio com defeito. É...,"1,0",23 de abril de 2020
...,...,...,...
4995,Estabilidade,"5,0",1 de fevereiro de 2021
4996,Gostei do produto!!!,"5,0",24 de fevereiro de 2021
4997,Suportou bem meu notebook que é pesado,"5,0",5 de maio de 2020
4998,"Estabilidade,e fácil de usar","5,0",30 de junho de 2021


In [ ]:
# Exportando o dataset contendo as reviews

df_reviews.to_csv('teste_projeto_NLP_5000.csv', decimal=',')